In [ ]:
import numpy as np
import argparse
import sys
import yaml
import csv
import scipy.constants as const


sys.path.insert(0, '/work/e89/shared/GCMC_Jan28/src')
from external_potentials import initialize_external_potentials

def conversion(T):
    return const.Boltzmann * T * const.Avogadro / 4184

def write_profile(filename, centers, mulocs):
    with open(filename, 'w', newline='') as csvfile:
        writer = csv.writer(csvfile, delimiter=' ')
        writer.writerow(["xbins", "muloc"])  # Include headers for your columns
        for center, muloc in zip(centers, mulocs):
            writer.writerow([f"{center:.4f}", f"{muloc:.16f}"])


def initialize_potentials(config, particle_type):
    """
    Function to initialize the external potentials based on the configuration.
    
    Parameters:
    - config (dict): Configuration dictionary.
    
    Returns:
    - beta (float): Inverse temperature (1/kB*T).
    - Vext (function): External potential function for the given particle type.
    - mu (float): Chemical potential for the given particle type.
    """
    external_potentials = initialize_external_potentials(config)
    beta = 1 / (config['kB'] * config['T'])

    Vext = external_potentials[particle_type]
    mu = config['particle_types'][particle_type]['mu'] * config['kB'] * config['T']

    return beta, Vext, mu, config['T']

if __name__ == '__main__':


    parser = argparse.ArgumentParser(
        description="Run GCMC simulations for short-ranged potentials."
    )

    parser.add_argument("-in", "--input_folder",
        required=False, type=str, default=".",
        help="the relative path to folder containing YAML input.",
    )

    args = parser.parse_args()

    input_folder = args.input_folder


    config_file = input_folder + '/' + 'input.yaml'
    # Load the configuration file
    with open(config_file, 'r') as file:
        config = yaml.safe_load(file)

    # Define bin edges
    bin_edges = x_positions = np.linspace(0, config['box_length_x'], 2001)

    bin_centers = (bin_edges[:-1] + bin_edges[1:]) / 2

    species_list = ["C", "O", "N", "X"]
    for species in species_list:
        # Initialize the external potentials and chemical potential
        beta, Vext, mu, T = initialize_potentials(config, species)

        # Compute Vext values and mu_loc profile
        xbins = np.array([bin_centers])
        
        Vext_values = Vext(xbins)

        muloc_profile = (Vext_values - mu) * beta

        a = np.append(bin_centers[::]-bin_centers[::][0], bin_centers[::][-1]-bin_centers[::][0]+bin_centers[::][0])


        force_profile = np.gradient(-muloc_profile, bin_centers) * conversion(T) 

        np.savetxt(input_folder + '/' + f"zbins.txt", a)
        np.savetxt(input_folder + '/' + f"force{species}.txt", force_profile)
        
        # Write the results to a file
        #name_file = input_folder + '/' + f"{species}_potential.out.test"
        #write_profile(name_file, bin_centers, -muloc_profile)

In [ ]:
import numpy as np
import argparse
import sys
import yaml
import csv
import scipy.constants as const

sys.path.insert(0, '/work/e89/shared/GCMC_Jan28/src')
from external_potentials import initialize_external_potentials


def conversion(T):
    return const.Boltzmann * T * const.Avogadro / 4184

def write_profile(filename, centers, mulocs):
    with open(filename, 'w', newline='') as csvfile:
        writer = csv.writer(csvfile, delimiter=' ')
        writer.writerow(["xbins", "muloc"])  # Include headers for your columns
        for center, muloc in zip(centers, mulocs):
            writer.writerow([f"{center:.4f}", f"{muloc:.16f}"])


def initialize_potentials(config, particle_type):
    """
    Function to initialize the external potentials based on the configuration.
    
    Parameters:
    - config (dict): Configuration dictionary.
    
    Returns:
    - beta (float): Inverse temperature (1/kB*T).
    - Vext (function): External potential function for the given particle type.
    - mu (float): Chemical potential for the given particle type.
    """
    external_potentials = initialize_external_potentials(config)
    beta = 1 / (config['kB'] * config['T'])

    Vext = external_potentials[particle_type]
    mu = config['particle_types'][particle_type]['mu'] * config['kB'] * config['T']

    return beta, Vext, mu, config['T']

if __name__ == '__main__':


    parser = argparse.ArgumentParser(
        description="Run GCMC simulations for short-ranged potentials."
    )

    parser.add_argument("-in", "--input_folder",
        required=False, type=str, default=".",
        help="the relative path to folder containing YAML input.",
    )

    args = parser.parse_args()

    input_folder = args.input_folder


    config_file = input_folder + '/' + 'input_nowalls.yaml'
    # Load the configuration file
    with open(config_file, 'r') as file:
        config = yaml.safe_load(file)

    # Define bin edges
    bin_edges = x_positions = np.linspace(0, config['box_length_x'], 2001)

    bin_centers = (bin_edges[:-1] + bin_edges[1:]) / 2

    species_list = ["C", "O"]
    for species in species_list:
        # Initialize the external potentials and chemical potential
        beta, Vext, mu, T = initialize_potentials(config, species)

        # Compute Vext values and mu_loc profile
        xbins = np.array([bin_centers])
        
        Vext_values = Vext(xbins)

        muloc_profile = (Vext_values - mu) * beta

        a = np.append(bin_centers[::]-bin_centers[::][0], bin_centers[::][-1]-bin_centers[::][0]+bin_centers[::][0])


        force_profile = np.gradient(-muloc_profile, bin_centers) * conversion(T)

        np.savetxt(input_folder + '/' + f"zbins.txt", a)
        np.savetxt(input_folder + '/' + f"force{species}.txt", force_profile)
        
        # Write the results to a file
        #name_file = input_folder + '/' + f"{species}_potential.out.test"
        #write_profile(name_file, bin_centers, -muloc_profile)